In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from qat.core.console import display

# 1. QLM

In [ ]:
from qat.lang.AQASM import Program

In [ ]:
def TensorialHadamard(qlmProgram, NumberOfQbits):
    from qat.lang.AQASM import H
    #Reservamos los qbits de entrada
    qbits = qlmProgram.qalloc(NumberOfQbits)
    for i in range(NumberOfQbits):
        qlmProgram.apply(H, qbits[i]) 
    return qlmProgram

In [ ]:
#Inicializamos el program
GrooverCircuit = Program()
GrooverCircuit = TensorialHadamard(GrooverCircuit, 2)
Groover = GrooverCircuit.to_circ()
%qatdisplay Groover

In [ ]:
Job4Groover = Groover.to_job()

In [ ]:
from qat.qpus import LinAlg
linalgqpu = LinAlg()

In [ ]:
result = linalgqpu.submit(Job4Groover)

In [ ]:
for sample in result:
    print("State %s amplitude %s" % (sample.state, sample.amplitude))

In [ ]:
#Aqui le decimos que mida
Job4Groover_02 = Groover.to_job(nbshots=10, aggregate_data=False)

In [ ]:
result = linalgqpu.submit(Job4Groover_02)

In [ ]:
# And use it to draw 10 samples
for sample in result:
    print("We measured the state {} (its probability is {} and its amplitude {})".format(
        sample.state, sample.probability, sample.amplitude))
    

### Oráculo

Voy a montar mi Hadamard Tensorial como una rutina

In [ ]:
def TensorialHaddamard(n):
    """
    Rutina que implementa una Puerta Hadamard n tensorial
    """
    from qat.lang.AQASM import QRoutine, H
    tensorialHadamard = QRoutine()
    for i in range(n):
        tensorialHadamard.apply(H,i)
    return tensorialHadamard

In [ ]:
#Inicializamos el program
NumberOfQbits = 2
#Definimos el circuito
Circuit = Program()
#Reservamos los qbits que queremos
qbits = Circuit.qalloc(NumberOfQbits)
#Aplicamos el Operado Hadamard n tensorial
Circuit.apply(TensorialHaddamard(NumberOfQbits),qbits)
Groover = Circuit.to_circ()
%qatdisplay Groover

In [ ]:
#Se implementa la Lotería que devuelve el número en decimal y binario
def Loteria(n):
    #Max number of elememnts I can generate
    N_max = 2**n
    print('N_max: {}'.format(N_max))
    #Generate winner number
    Winner = np.random.randint(N_max)
    print('Winner: {}'.format(Winner))
    stringFormat = '0{}b'.format(n)
    print(stringFormat)
    bWinner = format(Winner, stringFormat)
    return Winner, bWinner

In [ ]:
def Oracle(NumberOfQbits):
    from qat.lang.AQASM import QRoutine, X
    Winner = Loteria(NumberOfQbits)
    bWiner = Winner[1]
    Oracle = QRoutine()
    print(bWiner)
    #Loop over the winner binnary number in reverse order
    print('kk')
    for b, i in enumerate(list(bWiner[::-1])):
        #print((b,i))
        #if b == '0':
        Oracle.apply(X, i)
    return Oracle

In [ ]:
from qat.lang.AQASM import X, H, Program, CCNOT
#Inicializamos el program
NumberOfQbits = 4
#Definimos el circuito
Circuit = Program()
#Reservamos los qbits que queremos
qbits = Circuit.qalloc(NumberOfQbits)
qbitControl = Circuit.qalloc(1)

#Aplicamos el Operado Hadamard n tensorial
Circuit.apply(TensorialHaddamard(NumberOfQbits),qbits)
#Estado |-> en el qbit de Control
Circuit.apply(X, qbitControl)
Circuit.apply(H, qbitControl)

#Generamos el mnumero aleatorio
Winner = Loteria(NumberOfQbits)

for i, b in enumerate(Winner[1][::-1]):
    if b == '0':
        Circuit.apply(X, qbits[i])
Circuit.apply(CCNOT, qbits, qbitControl)
        
Groover = Circuit.to_circ()
%qatdisplay Groover



In [ ]:
from qat.lang.AQASM import X, H, Program, CCNOT

In [ ]:
NumberOfQbits = 2

from qat.lang.AQASM import QRoutine

rutina = QRoutine()
qbits = rutina.new_wires(NumberOfQbits)



#Para saber que hacer
control = NumberOfQbits % 2

if control == 0:
    NumberOfAncillaQbits =  NumberOfQbits//2
    qbits_ancilla = rutina.new_wires(NumberOfAncillaQbits)
    for i in range(NumberOfAncillaQbits):
        rutina.apply(CCNOT, qbits[2*i], qbits[2*i+1], qbits_ancilla[i])
    
else:
    NumberOfAncillaQbits =  NumberOfQbits//2
    qbits_ancilla = rutina.new_wires(NumberOfAncillaQbits+1)
    for i in range(NumberOfAncillaQbits):
        rutina.apply(CCNOT, qbits[2*i], qbits[2*i+1], qbits_ancilla[i])
    i=i+1
    rutina.apply(CNOT, qbits[2*i], qbits_ancilla[i])

    

rutina.set_ancillae(qbits_ancilla)

In [ ]:
def DoMidRoutine(NumberOfQbits):
    from qat.lang.AQASM import QRoutine
    rutina = QRoutine()
    qbits = rutina.new_wires(NumberOfQbits)
    
    Condition = True
    ListOfQbits = []
    while Condition:
    
        control = NumberOfQbits % 2
        if control == 0:
            NumberOfAncillaQbits =  NumberOfQbits//2
            qbits_ancilla = rutina.new_wires(NumberOfAncillaQbits)
            for i in range(NumberOfAncillaQbits):
                rutina.apply(CCNOT, qbits[2*i], qbits[2*i+1], qbits_ancilla[i])
            
        else:
            NumberOfAncillaQbits =  NumberOfQbits//2
            qbits_ancilla = rutina.new_wires(NumberOfAncillaQbits+1)
            for i in range(NumberOfAncillaQbits):
                rutina.apply(CCNOT, qbits[2*i], qbits[2*i+1], qbits_ancilla[i])
            i=i+1
            rutina.apply(CNOT, qbits[2*i], qbits_ancilla[i])    
            NumberOfAncillaQbits = NumberOfAncillaQbits+1
        rutina.set_ancillae(qbits_ancilla)  
        ListOfQbits.append(qbits)
        qbits=qbits_ancilla
        #bits=rutina.new_wires(NumberOfAncillaQbits)
        rutina.set_ancillae(qbits)
        NumberOfQbits = NumberOfQbits //2 
        if NumberOfQbits == 1:
            Condition = False
    return rutina

In [ ]:
NumberOfQbits = 7
Circuit = Program()
qb = Circuit.qalloc(NumberOfQbits)

Circuit.apply(DoMidRoutine(NumberOfQbits), qb)

Groover = Circuit.to_circ()
%qatdisplay Groover

In [ ]:
DoMidRoutine(NumberOfQbits).arity

In [ ]:
2//2

In [ ]:
Circuit.arcilla?

In [ ]:
int(NumberOfQbits/2)

In [ ]:
CCNOT.arity

In [ ]:
CCNOT?

In [ ]:
from qat.qpus import LinAlg
linalgqpu = LinAlg()
Job4Groover = Groover.to_job()

In [ ]:
result = linalgqpu.submit(Job4Groover)

In [ ]:
# And use it to draw 10 samples
for sample in result:
    print("We measured the state {} (its probability is {} and its amplitude {})".format(
        sample.state, sample.probability, sample.amplitude))

In [ ]:
from qat.lang.AQASM import X
for i, b in enumerate(Winner[1][::-1]):
    if b == '0':
        Circuit.apply(X, qbits[i])


In [ ]:
[i for i in Winner[1][::-1]]

In [ ]:
list(Winner[1][::-1])

In [ ]:
[i for i in Winner[1][::-1]]

In [ ]:
#Inicializamos el program
GrooverCircuit = Program()
n=2
GrooverCircuit = TensorialHadamard(GrooverCircuit, n)

Winner = Loteria(n)

Oracule(bWiner, qlmProgram, qbits)


Groover = GrooverCircuit.to_circ()
%qatdisplay Groover